# Reasoning With Probability and Pyro — Is My Model Good Enough?
- [talk](https://www.youtube.com/watch?v=5f-9xCuyZh4)
- [article](https://towardsdatascience.com/reasoning-with-probability-is-my-model-good-enough-1cfd27d5aed9)
- [Pyro Tutorials](http://pyro.ai/examples/#introductory-tutorials)

## Housekeeping

In [ ]:
# imports
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import pyro 
import torch 
import pyro
pyro.set_rng_seed(102563)

import pyro.distributions as dist 
import pyro.poutine as poutine

from pyro.infer.mcmc import HMC, MCMC

assert pyro.__version__.startswith('1.5.0')

## Data

In [ ]:
MNIST_X, MNIST_Y  = fetch_openml('mnist_784', version=1, return_X_y=True)
print(MNIST_X.shape)
print(MNIST_Y.shape)

#### Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(MNIST_X, MNIST_Y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

## Model

In [ ]:
my_tree = DecisionTreeClassifier(min_samples_leaf=2)
my_tree.fit(X_train, y_train)

In [ ]:
accuracy = my_tree.score(X_test, y_test)
print(f'Accuracy is {"{0:.2f}".format(accuracy*100)}%')

## Analysis

We have the luxury of having 14,000 labeled examples in the test set but what does this look like if we were to only have 100?

In [ ]:
_, X_100_test, _, y_100_test = train_test_split(X_test, y_test, test_size=(100/X_test.shape[0]), random_state=1)
print(X_100_test.shape, y_100_test.shape)

In [ ]:
accuracy = my_tree.score(X_100_test, y_100_test)
print(f'Accuracy is {"{0:.2f}".format(accuracy*100)}%')

Having faith in a model is inherently dependent on the number of samples that we are able to test it with.   
  
Our confidence in the test's set score increases as the size of the test set increases.  
  
With [pyro](http://docs.pyro.ai/en/1.1.0/index.html) we can numerically estimate our confidence and how much room for error we have. 

### Pyro
1. **Model Function:** 
    - simulates our underlying model (the process that results in correct or incorrect labels) which starts as our original, or prior, distribution
2. **Kernel:** 
    - measures the likelihood of the observed examples. This happens as the model function produces them. 
3. **Sampler:**
    - builds the updated, or posterior distribution

First we need to define our observations and generate an binary array that indicates if our predictions were correct or not.   
  
We then convert these to PyTorch tensors

In [ ]:
# define our observations
y_100_pred = my_tree.predict(X_100_test)
correctness_values_100 = y_100_pred == y_100_test   # An array of   
                                                    # observations 
                                                    # that shows
                                                    # if our 
                                                    # predictions 
                                                    # match the test
                                                    # or not

y_pred = my_tree.predict(X_test)
correctness_values = y_pred == y_test

correctness_values_100 = torch.from_numpy(correctness_values_100).float()
correctness_values = torch.from_numpy(correctness_values).float()

Now we define the **Model Function**.     
- It will accept our observations and try to sample from a prior distribution and calculate the likelihood of the prior based on our observations.   
- We use a uniform distribution here but any [torch distribution](https://pytorch.org/docs/master/distributions.html) is available depending on our knowledge of the underlying generating process of a value 
  
And the **Kernel** 
- It will then update the prior to a more likely posterior distribution. 

In [ ]:
def model(y):
    # Our observations are binary observations which come
    # from a Bernoulli distribution with some percent
    # "p" to be correct and (1-p) of being wrong
    
    # we want to estimate that p value. We start not
    # knowing anything about it, so our prior will be
    # a uniform distribution from 0.0 to 1.0
    underlying_p = pyro.sample("p", dist.Uniform(0.0, 1.0)) 

    # for each observation
    for i in range(len(y)):

        # our hidden distribution
        y_hidden_dist = dist.Bernoulli(underlying_p)

        # now sample from our distribution conditioned on our 
        # observation
        y_real = pyro.sample("obs_{}".format(i), 
                             y_hidden_dist, 
                             obs = y[i])

In [ ]:
pyro.sample??

In [ ]:
# First clear the old values of all our stored parameters
pyro.clear_param_store()

# the kernel we will use
hmc_kernel = HMC(model,
                 step_size = 0.1)


# the sampler which will run the kernel
mcmc = MCMC(hmc_kernel, num_samples=100, warmup_steps=100)

# the .run method accepts as parameter the same parameters our model function uses
mcmc.run(correctness_values_100)

In [ ]:
sample_dict = mcmc.get_samples(num_samples=5000)

plt.figure(figsize=(10,7))
sns.distplot(sample_dict['p'].numpy(), color="orange");
plt.xlabel("Observed probability value")
plt.ylabel("Observed frequency")
plt.show();

mcmc.summary(prob=0.95)